<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Create Player dataset

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

num_of_players = len(df) #250

projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

projections_df.info()
print(projections_df.value_counts('POS'))
print(projections_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  948 non-null    object 
 1   Bye     948 non-null    int64  
 2   POS     948 non-null    object 
 3   AVG     948 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 29.8+ KB
POS
WR     335
RB     224
TE     166
QB     127
K       64
DST     32
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12      

# Create custom projections
Maybe delete later when real projections available

In [42]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def projection_base(avg, pos, max_val=22, min_val=7, k=75, c=1.5):
    proj = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        proj += 8  # QB-Bonus
    elif pos == "K":
        proj -= 2  # K-Penalty
    elif pos == "DST":
        proj -= 3  # DST-Penalty
    return proj

final_projections = []
for _, row in projections_df.iterrows():
    base_score = projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = projections_df[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

f["TTL"] = f[weekly_columns].sum(axis=1)

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(50)

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,TTL
0,Christian McCaffrey,9,RB,1.0,20.476797,20.962507,21.051979,23.078936,20.180244,20.333077,...,0.000000,19.623729,22.802711,26.375561,20.969082,22.601985,19.916575,24.556737,20.796849,346.788113
1,CeeDee Lamb,7,WR,2.6,23.822432,21.728975,23.398425,23.620444,24.978570,22.152295,...,24.316029,19.520663,25.227487,24.473798,21.291789,22.574771,25.648025,21.652975,22.571993,368.784395
2,Tyreek Hill,6,WR,3.2,17.848110,19.902587,21.888030,22.192247,19.751496,0.000000,...,24.881948,20.972536,22.373542,24.949428,21.631161,20.161670,25.489963,19.882293,22.251443,349.935506
3,Bijan Robinson,12,RB,5.0,23.425225,22.519100,20.411284,24.047798,23.775272,22.866136,...,24.863875,20.014159,20.463757,0.000000,22.560606,20.815874,20.627585,22.345666,21.804229,349.915293
4,Breece Hall,12,RB,5.4,16.230513,23.157778,19.896443,18.866995,23.752487,20.220869,...,22.133835,18.789467,23.021292,0.000000,22.814128,20.849065,22.621688,24.570620,21.945344,338.095183
5,Amon-Ra St. Brown,5,WR,6.2,23.215922,23.306461,20.867856,23.181856,0.000000,26.427558,...,20.635452,21.150125,20.858881,19.887461,21.303941,25.005494,22.206703,24.213428,23.626271,357.988015
6,Ja'Marr Chase,12,WR,6.6,20.069042,18.958140,17.905881,23.654144,24.254699,21.945913,...,19.701514,19.388894,17.695117,0.000000,22.047127,20.595939,20.167048,18.815448,22.198967,324.977221
7,Justin Jefferson,6,WR,7.0,23.542688,20.997435,21.608354,18.876315,19.821965,0.000000,...,19.185059,22.930415,19.816210,21.821442,22.429205,23.511472,23.215875,20.500146,20.701775,346.708593
8,Saquon Barkley,5,RB,9.2,21.454075,18.338797,19.696610,21.873964,0.000000,18.296352,...,20.895351,21.144141,22.397902,21.244617,20.400235,23.439791,23.182152,20.625154,23.445284,336.774727
9,A.J. Brown,5,WR,10.2,22.302765,22.713608,21.728816,23.242994,0.000000,21.663087,...,20.091120,22.241414,20.480764,22.818922,18.390219,23.357457,23.068125,22.790460,18.690950,344.121855


In [ ]:
# --- 1. Parameter (aus deinem paste-2.txt) ---
players        = projections_df['Player'].tolist()
positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks          = list(range(1,18))
pos            = dict(zip(projections_df['Player'], projections_df['POS']))
pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
beta           = {t:125.0 for t in weeks}
gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
alpha, lambda_0, lambda_1, lambda_2 = 1.0, 1, 100, 150
df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
topk_pct = 0.005
min_pos_req = pos_limit


# --- 2. Teams, DM-Team und Snake-Draft ---
num_teams      = 12
teams          = [f"Team {i+1}" for i in range(num_teams)]
DM_team        = "Team 3"
num_rounds     = 15
draft_order    = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) verbleibende Spieler neu sortieren
    rem  = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) solange Defizite bestehen, aus allen rem dieser Position picken
    if needed:
        # Kandidaten aller benötigten Positionen
        candidates = [p for p in rem if pos[p] in needed]
        # begrenze auf Top-k, falls mehr Kandidaten vorhanden
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: zufällig aus Top-k aller Positionen
    return random.choice(rem[:topk])

# --- 3. Initialisierung ---
rosters        = {tm: [] for tm in teams}
available      = set(players)
draft_log      = []

# --- 4. Hauptschleife über alle Picks ---
for pick_idx, team in enumerate(draft_order, start=1):
    # 4.1 Ranking der verbleibenden Spieler aktualisieren
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
    Rk  = {p: i+1 for i,p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])


    # 4.2 DM-Pick via MIP
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # Variablen nur für verbleibende Spieler
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
              for i in player_vars}
        x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
             for i in rem for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # Fixiere vergangene Picks
        for p in rosters[team]:
            m += y[p] == 1

        # Objective
        m.objective = (
            lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
          + lambda_1 * xsum(z[t] for t in weeks[:15])
          + lambda_2 * xsum(z[t] for t in weeks[15:])
        )

        # Position‐ und Roster‐Constraints
        for j in positions:
            # Müssen mindestens gamma[j] ziehen
            m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
            for t in weeks:
                m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]
        # Roster Anforderungen
        for pos_name, req in min_pos_req.items():
            m.add_constr(
                xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
                name=f"min_roster_{pos_name}")
        # maximale Picks pro Team
        m += xsum(y[i] for i in rem) <= num_rounds
        # nur gedraftete Spieler in der Week‐Lineup
        for i in rem:
            for t in weeks:
                m += x[i,t] <= y[i]
        # Win‐Indicator
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

        # Robuste Draft‐Constraint (1b) – grob implementiert
        n_k = pick_idx
        for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
            top_cut = int(alpha*(future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i,r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

        m.optimize()
        # gewählten Spieler extrahieren
        chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
    else:
        pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=topk_pct)

    # 4.4 Update
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx, "Team": team, "Player": pick,
        "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
    })

# --- 5. Ergebnis als DataFrame ---
df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1+num_teams*2))


In [9]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


In [10]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

POS,DST,K,QB,RB,TE,WR
Team,,,,,,
Team 1,2,1,3,2,2,5
Team 10,1,2,2,4,1,5
Team 11,1,1,3,3,1,6
Team 12,1,1,2,4,2,5
Team 2,1,1,1,4,2,6
Team 3,2,1,2,6,1,3
Team 4,1,2,1,4,1,6
Team 5,1,1,1,5,1,6
Team 6,1,1,1,5,2,5


In [11]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df['Pick Info'] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f"=== {team} ===")
    print(df)
    print()


=== Team 1 ===
                  Player          Pick Info  POS
0       Baltimore Ravens     Round 1 Pick 1  DST
23        Pat Freiermuth    Round 2 Pick 24   TE
24          Malik Nabers    Round 3 Pick 25   WR
47             Cole Kmet    Round 4 Pick 48   TE
48         Calvin Ridley    Round 5 Pick 49   WR
71           Jordan Love    Round 6 Pick 72   QB
72        Deshaun Watson    Round 7 Pick 73   QB
95        Terry McLaurin    Round 8 Pick 96   WR
96    Brian Robinson Jr.    Round 9 Pick 97   RB
119         Keon Coleman  Round 10 Pick 120   WR
120        Aaron Rodgers  Round 11 Pick 121   QB
143     Brian Thomas Jr.  Round 12 Pick 144   WR
144         Younghoe Koo  Round 13 Pick 145    K
167  Philadelphia Eagles  Round 14 Pick 168  DST
168        Jaylen Wright  Round 15 Pick 169   RB

=== Team 2 ===
              Player          Pick Info  POS
1        Tyreek Hill     Round 1 Pick 2   WR
22   Jonathan Taylor    Round 2 Pick 23   RB
25       Josh Jacobs    Round 3 Pick 26   RB
46   